# Optimisation de l'optique pour une résonance du troisième ordre

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xtrack as xt

%config InlineBackend.figure_format = "retina"

### Chargement de la maille exportée précédemment

In [ ]:
env = xt.Environment.from_json("pimm.json")
ring = env["ring"]

In [ ]:
# Attribuons de faibles valeurs pour obtenir un Twiss stable
env["kqfa"] = 1e-2
env["kqfb"] = 1e-2
env["kqd"] = -2e-2

In [ ]:
twiss = ring.twiss4d(compute_chromatic_properties=False)
plot = twiss.plot(grid=False, lattice=False)

### Ajustement des nombres d'onde

In [ ]:
opt_tune = ring.match(
    solve=False,  # <- prépare uniquement
    vary=[
        xt.Vary("kqfa", limits=(0, 10), step=1e-3),
        xt.Vary("kqfb", limits=(0, 10), step=1e-3),
        xt.Vary("kqd", limits=(-10, 0), step=1e-3),
    ],
    targets=[  # à proximité d'une résonance du 3ème ordre (3qx~5, 2qx+qy~5, qx+2qy~5)
        xt.TargetSet(qx=1.665, qy=1.72, tol=1e-4),
    ],
    compute_chromatic_properties=False,
    method="4d",
)

In [ ]:
opt_tune.target_status()

In [ ]:
opt_tune.run_jacobian(30)

In [ ]:
opt_tune.target_status()

In [ ]:
opt_tune.vary_status()

In [ ]:
# Inspection de l'optique
twiss = ring.twiss4d()
plot = twiss.plot(grid=False, lattice=False)
plot.ylim(left_hi=30, right_lo=-10, right_hi=10);

### Addition d'une contrainte de dispersion nulle au centre des sections droites longues

In [ ]:
opt_disp = opt_tune.clone(  # cloner l'optimiseur préserve les contraintes précédentes
    add_targets=[
        xt.TargetSet(dx=0, at="mid.lss.0"),
        xt.TargetSet(dx=0, at="mid.lss.1"),
    ]
)
opt_disp.target_status()

In [ ]:
opt_disp.run_simplex(100)  # Nelder-Mead optimisation algorithm

In [ ]:
# Inspect the optics
twiss = ring.twiss4d()
plot = twiss.plot(grid=False, lattice=False)
plot.ylim(left_hi=30, right_lo=-10, right_hi=10)

## "Correction" de la chromaticité

In [ ]:
opt_chrom = ring.match(
    solve=False,
    method="4d",
    vary=xt.VaryList(["ksf", "ksd"], step=1e-3),
    targets=xt.TargetSet(dqx=-0.01, dqy=-0.01, tol=1e-3),
)

In [ ]:
opt_chrom.run_ls_dogbox(30)  # Least squares, dobgox algorithm
opt_chrom.target_status()

## Déformation de l'espace des phases lors de l'excitation de la résonance

In [ ]:
# Alimentation du sextupôle d'extraction
env["kse"] = 6

In [ ]:
# Génération de 20 particules sur l'axe horizontal
x_gen = np.linspace(0, 0.5e-2, 20)
parts = ring.build_particles(x=x_gen, px=0, y=0, py=0, zeta=0, delta=0)

# Tracking des particules pour 1000 tours
ring.track(parts, num_turns=1000, turn_by_turn_monitor=True)
record = ring.record_last_track

In [ ]:
plt.figure()
plt.plot(record.x.T, record.px.T, ".", markersize=1, color="C0")
plt.xlim(-2e-2, 2e-2)
plt.ylim(-2e-3, 2e-3)
plt.xlabel(r"$x$ [m]")
plt.ylabel(r"$p_x$")
plt.title("Espace des phases horizontal")
plt.subplots_adjust(left=0.15)
plt.show()

### Export des alimentations en fichier JSON

In [ ]:
# Valeurs des résultats des optimiseurs
quad_strengths: dict[str, float] = opt_disp.get_knob_values()
sext_strengths: dict[str, float] = opt_chrom.get_knob_values()

In [ ]:
# Fusion et export en fichier
strengths = quad_strengths | sext_strengths
xt.json.dump(strengths, "pimm_strengths.json")

---